# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 9 2022 11:32AM,243768,19,ADV80004746,"AdvaGen Pharma, Ltd",Released
1,Aug 9 2022 11:32AM,243768,19,ADV80004748,"AdvaGen Pharma, Ltd",Released
2,Aug 9 2022 11:32AM,243768,19,ADV80004749,"AdvaGen Pharma, Ltd",Released
3,Aug 9 2022 11:32AM,243768,19,ADV80004750,"AdvaGen Pharma, Ltd",Released
4,Aug 9 2022 11:32AM,243768,19,ADV80004751,"AdvaGen Pharma, Ltd",Released
5,Aug 9 2022 11:32AM,243768,19,ADV80004753,"AdvaGen Pharma, Ltd",Released
6,Aug 9 2022 11:32AM,243768,19,ADV80004754,"AdvaGen Pharma, Ltd",Released
7,Aug 9 2022 11:32AM,243768,19,ADV80004755,"AdvaGen Pharma, Ltd",Released
8,Aug 9 2022 11:32AM,243768,19,ADV80004756,"AdvaGen Pharma, Ltd",Released
9,Aug 9 2022 11:32AM,243768,19,ADV80004758,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,243765,Released,1
33,243766,Released,4
34,243767,Released,9
35,243768,Executing,1
36,243768,Released,24


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243764,NaN,NaN,14.0
243765,NaN,NaN,1.0
243766,NaN,NaN,4.0
243767,NaN,NaN,9.0
243768,NaN,1.0,24.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243677,0.0,1.0,0.0
243682,0.0,0.0,1.0
243687,13.0,1.0,1.0
243702,0.0,1.0,0.0
243721,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243677,0,1,0
243682,0,0,1
243687,13,1,1
243702,0,1,0
243721,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243677,0,1,0
1,243682,0,0,1
2,243687,13,1,1
3,243702,0,1,0
4,243721,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243677,,1,
1,243682,,,1
2,243687,13,1,1
3,243702,,1,
4,243721,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd"
25,Aug 9 2022 11:00AM,243767,10,MedStone Pharma LLC
34,Aug 9 2022 10:59AM,243766,10,Emerginnova
38,Aug 9 2022 10:47AM,243764,10,Beach Patient Assistance
52,Aug 9 2022 10:46AM,243765,19,"NAPP Technologies, LLC"
53,Aug 9 2022 10:15AM,243758,21,"NBTY Global, Inc."
54,Aug 9 2022 10:09AM,243757,15,"Mizner Bioscience, LLC"
56,Aug 9 2022 10:09AM,243755,15,"Person & Covey, Inc."
64,Aug 9 2022 10:03AM,243754,10,Eywa Pharma Inc.
95,Aug 9 2022 9:59AM,243753,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd",,1,24
1,Aug 9 2022 11:00AM,243767,10,MedStone Pharma LLC,,,9
2,Aug 9 2022 10:59AM,243766,10,Emerginnova,,,4
3,Aug 9 2022 10:47AM,243764,10,Beach Patient Assistance,,,14
4,Aug 9 2022 10:46AM,243765,19,"NAPP Technologies, LLC",,,1
5,Aug 9 2022 10:15AM,243758,21,"NBTY Global, Inc.",,1,
6,Aug 9 2022 10:09AM,243757,15,"Mizner Bioscience, LLC",,,2
7,Aug 9 2022 10:09AM,243755,15,"Person & Covey, Inc.",,,8
8,Aug 9 2022 10:03AM,243754,10,Eywa Pharma Inc.,,,31
9,Aug 9 2022 9:59AM,243753,10,"Methapharm, Inc.",,1,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd",24,1,
1,Aug 9 2022 11:00AM,243767,10,MedStone Pharma LLC,9,,
2,Aug 9 2022 10:59AM,243766,10,Emerginnova,4,,
3,Aug 9 2022 10:47AM,243764,10,Beach Patient Assistance,14,,
4,Aug 9 2022 10:46AM,243765,19,"NAPP Technologies, LLC",1,,
5,Aug 9 2022 10:15AM,243758,21,"NBTY Global, Inc.",,1,
6,Aug 9 2022 10:09AM,243757,15,"Mizner Bioscience, LLC",2,,
7,Aug 9 2022 10:09AM,243755,15,"Person & Covey, Inc.",8,,
8,Aug 9 2022 10:03AM,243754,10,Eywa Pharma Inc.,31,,
9,Aug 9 2022 9:59AM,243753,10,"Methapharm, Inc.",1,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd",24,1,
1,Aug 9 2022 11:00AM,243767,10,MedStone Pharma LLC,9,,
2,Aug 9 2022 10:59AM,243766,10,Emerginnova,4,,
3,Aug 9 2022 10:47AM,243764,10,Beach Patient Assistance,14,,
4,Aug 9 2022 10:46AM,243765,19,"NAPP Technologies, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd",24.0,1.0,NaN
1,Aug 9 2022 11:00AM,243767,10,MedStone Pharma LLC,9.0,NaN,NaN
2,Aug 9 2022 10:59AM,243766,10,Emerginnova,4.0,NaN,NaN
3,Aug 9 2022 10:47AM,243764,10,Beach Patient Assistance,14.0,NaN,NaN
4,Aug 9 2022 10:46AM,243765,19,"NAPP Technologies, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 11:32AM,243768,19,"AdvaGen Pharma, Ltd",24.0,1.0,0.0
1,Aug 9 2022 11:00AM,243767,10,MedStone Pharma LLC,9.0,0.0,0.0
2,Aug 9 2022 10:59AM,243766,10,Emerginnova,4.0,0.0,0.0
3,Aug 9 2022 10:47AM,243764,10,Beach Patient Assistance,14.0,0.0,0.0
4,Aug 9 2022 10:46AM,243765,19,"NAPP Technologies, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3656018,126.0,12.0,6.0
15,974959,113.0,44.0,0.0
19,975022,26.0,2.0,0.0
20,487431,8.0,9.0,13.0
21,731201,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3656018,126.0,12.0,6.0
1,15,974959,113.0,44.0,0.0
2,19,975022,26.0,2.0,0.0
3,20,487431,8.0,9.0,13.0
4,21,731201,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,126.0,12.0,6.0
1,15,113.0,44.0,0.0
2,19,26.0,2.0,0.0
3,20,8.0,9.0,13.0
4,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,126.0
1,15,Released,113.0
2,19,Released,26.0
3,20,Released,8.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,6.0,0.0,0.0,13.0,0.0
Executing,12.0,44.0,2.0,9.0,1.0
Released,126.0,113.0,26.0,8.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,6.0,0.0,0.0,13.0,0.0
1,Executing,12.0,44.0,2.0,9.0,1.0
2,Released,126.0,113.0,26.0,8.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,6.0,0.0,0.0,13.0,0.0
1,Executing,12.0,44.0,2.0,9.0,1.0
2,Released,126.0,113.0,26.0,8.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()